<a href="https://colab.research.google.com/github/shatakshigarg/LeanIn_July2021_NLP_ShatakshiGarg/blob/main/Tweet_Act_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tweet Act Classification**

In [73]:
!wget https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt
!cp glove.840B.300d.sst.txt sample_data/glove.840B.300d.sst.txt

--2021-09-01 20:02:35--  https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53933315 (51M) [text/plain]
Saving to: ‘glove.840B.300d.sst.txt’

glove.840B.300d.sst 100%[===================>]  51.43M   157MB/s    in 0.3s    

2021-09-01 20:02:36 (157 MB/s) - ‘glove.840B.300d.sst.txt’ saved [53933315/53933315]



# **Import Moldules**

In [74]:
import pandas as pd
import numpy as np
import os
import pickle
import numpy as np
import re
import itertools
from collections import Counter
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import *
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from numpy import array
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing import sequence
from keras.regularizers import *
import tensorflow as tf
from keras.preprocessing.text import *
from keras.preprocessing.sequence import pad_sequences
import timeit
from tensorflow.keras.optimizers import Adam

# **Import Important Functions**

In [75]:
def dumpPickle(path,object_to_dump):
	f=open(path,'wb')
	pickle.dump(object_to_dump,f)
	f.close()

#Reads an object stores at 'path'
def readPickle(path):
	f=open(path,'rb')
	ob_to_load=pickle.load(f)
	f.close()
	return ob_to_load

def load_create_embedding_matrix(word_index,vocab_size,emb_dim,emb_path,emb_pickle_path=False,save=False,saveName=None):
	if not emb_pickle_path:
		embedding_dict={}
		f=open(emb_path,'rb')
		for line in f:
			fields=line.split()
			word=fields[0]
			w_e=np.asarray(fields[1:],dtype='float32')
			embedding_dict[word]=w_e
		f.close()
		allembs=np.stack(embedding_dict.values())
		emb_mean,emb_std=allembs.mean(),allembs.std()
		embedding_matrix=np.random.normal(emb_mean,emb_std,(vocab_size,emb_dim))
		for word,index in word_index.items():
		
			vector=embedding_dict.get(word)
			if vector is not None:
				embedding_matrix[index]=vector
		if save:
			dumpPickle(saveName,embedding_matrix)
	else:
		f=open(emb_pickle_path,'rb')
		embedding_matrix=pickle.load(f)
		f.close()
	return embedding_matrix
  
def load_create_tokenizer(train_data,tok_path=None,savetokenizer=False):
	if tok_path == None:
		tokenizer=Tokenizer()
		tokenizer.fit_on_texts(train_data)
		len(tokenizer.word_index)
		if savetokenizer:
			dumpPickle('./New_Tokenizer.tkn',tokenizer)
	else:
		tokenizer=readPickle(tok_path)
	return tokenizer

def load_create_padded_data(X_train,savetokenizer,padPath=None,isPaddingDone=False,maxlen=None,tokenizer_path=None,save_new_padded_data=False,path_for_new_data=False):
	
	if not isPaddingDone:
		tokenizer=load_create_tokenizer(X_train,tok_path=tokenizer_path,savetokenizer=savetokenizer)
		
		X_train=tokenizer.texts_to_sequences(X_train)
		X_train=pad_sequences(X_train,maxlen=maxlen)
		if save_new_padded_data:
			dumpPickle(path_for_new_data,X_train)
	else:
		X_train=readPickle(padPath)
	return X_train

# **Load Dataset**

In [76]:
tweet_df=pd.read_csv("/content/TWEET_DATA_IEEE_TCSS_-_Sheet1.csv",names=["TWEET","LABEL"])
tweet_df.head()

,TWEET,LABEL
0,RT @Bipartisanism: Katie Couric just schooled ...,STM
1,RT @stevesilberman: If you missed it: Sick bur...,EXP
2,@jk_rowling & the Never-Ending Story: With a #...,QUE
3,RT @somebadideas: The emotional & personal eff...,EXP
4,RT @jk_rowling: A 9-year-old Nigerian girl has...,STM


In [77]:
text = tweet_df['TWEET'].tolist()

In [78]:
max1=0

for i in range(0,len(text)):
	data=text[i].split(" ")
	max2=len(data)
	if(max2>max1):
		max1=max2

sequence_length = max1

In [79]:
train, test = train_test_split(text, test_size = 0.2)

# **Create Padded-Data & Embedding Matrix**

In [80]:
print("creating data")


tokenizer=load_create_tokenizer(train,None,True)
X_train=load_create_padded_data(X_train=train,savetokenizer=False,isPaddingDone=False,maxlen=sequence_length,tokenizer_path='./New_Tokenizer.tkn')
X_test=load_create_padded_data(X_train=test,savetokenizer=False,isPaddingDone=False,maxlen=sequence_length,tokenizer_path='./New_Tokenizer.tkn')
word_index=tokenizer.word_index
embedding_matrix=load_create_embedding_matrix(word_index,len(word_index)+1,300,'./glove.840B.300d.sst.txt',False,True,'./Emb_Mat.mat')

print("data created")

creating data


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


data created


In [81]:
a=len(word_index)
print(a)

15381


In [82]:
X_train.shape

(6000, 57)

In [83]:
X_test.shape

(1500, 57)

In [84]:
X_train[125]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  253,
          2,  561,  694,  414, 6512,  624,   52,   64, 2446,  290,  118,
        845,  302,   12,  120,  488,   49, 4133,  488,   77, 3063,    9,
        489,   28], dtype=int32)

In [85]:
embedding_matrix.shape

(15382, 300)

In [86]:
labels = tweet_df['LABEL'].tolist()
train_l, test_l = train_test_split(labels, test_size = 0.2)

# **One-Hot Encoded Labels**

In [87]:
lbl_dict={}
index=0
for dial_lbls in train_l:
	if dial_lbls not in lbl_dict:
		lbl_dict[dial_lbls]=index
		index=index+1

print((lbl_dict))



def create_label(label):
	
    Y=[]
    for i in label:
    	xxx=np.zeros(len(lbl_dict))  
    	j=lbl_dict.get(i)
    	xxx[j]=1
    	Y.append(xxx)
    return Y

label = train_l
Y_train = create_label(label)

label = test_l
Y_test = create_label(label)

y_train=np.array([i for i in Y_train])
y_test=np.array([i for i in Y_test])

embedding_dim = 300

{'EXP': 0, 'STM': 1, 'OTH': 2, 'THT': 3, 'QUE': 4, 'REQ': 5, 'SUG': 6}


In [88]:
y_train.shape

(6000, 7)

# **CNN**

In [89]:
print("Creating Model...")

inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=len(word_index)+1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=sequence_length)(inputs)
conv_0 = Conv1D(filters=32, kernel_size=3, padding='same', kernel_initializer='normal', activation='relu')(embedding)
maxpool_0 = MaxPooling1D(pool_size=2)(conv_0)
dropout = Dropout(0.1)(maxpool_0)
conv_1 = Conv1D(filters=32, kernel_size=3, padding='same', kernel_initializer='normal', activation='relu')(dropout)
maxpool_1 = MaxPooling1D(pool_size=2)(conv_1)
flatten = Flatten()(maxpool_1)
predictions = Dense(1000, activation='relu')(flatten)

predictions1 = Dense(len(lbl_dict), activation='softmax')(predictions)
adam = Adam(lr=0.01, decay=0.3)
model = Model(inputs=inputs, outputs=predictions1)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=4, batch_size=64, validation_data=(X_test, y_test))


predicted=model.predict(X_test)


Creating Model...
Epoch 1/4


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


94/94 [==============================] - 10s 100ms/step - loss: 1.6159 - accuracy: 0.5033 - val_loss: 1.3861 - val_accuracy: 0.5567
Epoch 2/4
94/94 [==============================] - 9s 92ms/step - loss: 1.4174 - accuracy: 0.5218 - val_loss: 1.3898 - val_accuracy: 0.5567
Epoch 3/4
94/94 [==============================] - 9s 92ms/step - loss: 1.4128 - accuracy: 0.5218 - val_loss: 1.3801 - val_accuracy: 0.5567
Epoch 4/4
94/94 [==============================] - 9s 95ms/step - loss: 1.4107 - accuracy: 0.5218 - val_loss: 1.3846 - val_accuracy: 0.5567


In [90]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 57)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 57, 300)           4614600   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 57, 32)            28832     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 28, 32)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 28, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 28, 32)            3104      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 14, 32)            0   

In [91]:
Y_predicted=[]
for i in predicted:
    pos=i.argmax()
    Y_predicted.append(pos)
    
    
    
Y_test=[]
for i in y_test:
    pos=i.argmax()
    Y_test.append(pos)
    
    
    
from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


print(accuracy_score(Y_test, Y_predicted))
print(classification_report(Y_test, Y_predicted,digits=4,))
print(confusion_matrix(Y_test, Y_predicted))

0.5566666666666666
              precision    recall  f1-score   support

           0     0.5567    1.0000    0.7152       835
           1     0.0000    0.0000    0.0000       302
           2     0.0000    0.0000    0.0000        73
           3     0.0000    0.0000    0.0000        62
           4     0.0000    0.0000    0.0000       118
           5     0.0000    0.0000    0.0000        37
           6     0.0000    0.0000    0.0000        73

    accuracy                         0.5567      1500
   macro avg     0.0795    0.1429    0.1022      1500
weighted avg     0.3099    0.5567    0.3981      1500

[[835   0   0   0   0   0   0]
 [302   0   0   0   0   0   0]
 [ 73   0   0   0   0   0   0]
 [ 62   0   0   0   0   0   0]
 [118   0   0   0   0   0   0]
 [ 37   0   0   0   0   0   0]
 [ 73   0   0   0   0   0   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#**Bi-LSTM**

In [92]:
 from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [93]:
 n_unique_words = a+1
 maxlen = 200
 batch_size = 72

In [94]:
 model = Sequential()
 model.add(Embedding(n_unique_words, 300, input_length=maxlen))
 model.add(Bidirectional(LSTM(72)))
 model.add(Dropout(0.5))
 model.add(Dense(7, activation='sigmoid'))
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [95]:
 mymodel=model.fit(X_train, y_train, batch_size=batch_size, epochs=4, validation_data=(X_test, y_test))

Epoch 1/4
84/84 [==============================] - 29s 303ms/step - loss: 0.3651 - accuracy: 0.4932 - val_loss: 0.3071 - val_accuracy: 0.5567
Epoch 2/4
84/84 [==============================] - 25s 298ms/step - loss: 0.3221 - accuracy: 0.5193 - val_loss: 0.3073 - val_accuracy: 0.5567
Epoch 3/4
84/84 [==============================] - 25s 297ms/step - loss: 0.3007 - accuracy: 0.5442 - val_loss: 0.3255 - val_accuracy: 0.5100
Epoch 4/4
84/84 [==============================] - 25s 292ms/step - loss: 0.2379 - accuracy: 0.6578 - val_loss: 0.3747 - val_accuracy: 0.4947


In [96]:
predicted_bilstm=model.predict(X_test)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 300)          4614600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 144)               214848    
_________________________________________________________________
dropout_3 (Dropout)          (None, 144)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 1015      
Total params: 4,830,463
Trainable params: 4,830,463
Non-trainable params: 0
_________________________________________________________________


In [97]:
Y_predicted=[]
for i in predicted_bilstm:
    pos=i.argmax()
    Y_predicted.append(pos)
    
    
    
Y_test=[]
for i in y_test:
    pos=i.argmax()
    Y_test.append(pos)
    
    
    
from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


print(accuracy_score(Y_test, Y_predicted))
print(classification_report(Y_test, Y_predicted,digits=4,))
print(confusion_matrix(Y_test, Y_predicted))

0.49466666666666664
              precision    recall  f1-score   support

           0     0.5575    0.8132    0.6615       835
           1     0.2469    0.1987    0.2202       302
           2     0.0000    0.0000    0.0000        73
           3     0.0000    0.0000    0.0000        62
           4     0.0857    0.0254    0.0392       118
           5     0.0000    0.0000    0.0000        37
           6     0.0000    0.0000    0.0000        73

    accuracy                         0.4947      1500
   macro avg     0.1272    0.1482    0.1316      1500
weighted avg     0.3668    0.4947    0.4156      1500

[[679 133   0   0  22   0   1]
 [233  60   0   0   6   0   3]
 [ 57  15   0   0   1   0   0]
 [ 54   7   0   0   1   0   0]
 [101  14   0   0   3   0   0]
 [ 31   6   0   0   0   0   0]
 [ 63   8   0   0   2   0   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
